In [4]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
video_json_1 = open('video_api_data_full\\full_videos_output_1_50.json')
video_json_2 = open('video_api_data_full\\full_videos_output_51_100.json')

In [6]:
video_data_1 = json.load(video_json_1)
video_data_2 = json.load(video_json_2)

In [7]:
columns = [
    ''
]
df 

pd.DataFrame.from_dict(data)()

{'items': [{'id': 'TJ2ifmkGGus',
   'snippet': {'publishedAt': '2023-01-28T21:00:00Z',
    'title': '1,000 Blind People See For The First Time'},
   'contentDetails': {'duration': 'PT8M1S'},
   'statistics': {'viewCount': '101200476',
    'likeCount': '8229283',
    'favoriteCount': '0',
    'commentCount': '343887'}},
  {'id': '7IKab3HcfFk',
   'snippet': {'publishedAt': '2022-12-24T20:59:59Z',
    'title': 'I Survived 50 Hours In Antarctica'},
   'contentDetails': {'duration': 'PT12M10S'},
   'statistics': {'viewCount': '92959825',
    'likeCount': '3884374',
    'favoriteCount': '0',
    'commentCount': '148754'}},
  {'id': 'h5NvTTOlOtI',
   'snippet': {'publishedAt': '2022-12-10T21:00:01Z',
    'title': 'Hydraulic Press Vs Lamborghini'},
   'contentDetails': {'duration': 'PT10M25S'},
   'statistics': {'viewCount': '110172252',
    'likeCount': '4380170',
    'favoriteCount': '0',
    'commentCount': '126247'}},
  {'id': '0CTp1a-aCUM',
   'snippet': {'publishedAt': '2022-12-03T21:00